In [3]:
# Simulated annealing C-style
# imports
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [7]:
# Read data functions
def string_to_spin(string):
    """
    Convert string of 0 and 1 to a np array of spins (-1, 1):
    0 -> -1
    1 -> +1
    :param string: String: original string.
    :return: np.array: Spins.
    """
    return (np.array(list(map(int, list(string)))) * 2 - 1)

def read_configurations(name, n):
    """
    Reads a file and returns a np array with the spin configuration.
    :param name: String: Name of the file.
    :param n: Int: Number of spins.
    :return: np.array: Matrix of shape (Data_number, num_spins).
    """
    spins = np.array([])
    filepath = name
    with open(filepath) as fp:
        for line in tqdm_notebook(fp.readlines()):
            spins = np.append(spins, string_to_spin(line.rstrip()))
    return np.reshape(spins, (int(len(spins)/n), n)).astype('int')

def read_j_coef(name, n):
    """
    Reads a file and returns a np array with the j coefficients.
    :param name: String: Name of the file.
    :param n: Int: Number of spins.
    :return: np.array: Array of shape 2*n.
    """
    j = np.zeros((n, 2))
    filepath = name
    with open(filepath) as fp:
        for line in tqdm_notebook(fp.readlines()):
            list_line = line.split()
            if len(list_line) == 2:
                el = list_line[-1]
                list_line[-1] = el[0]
                list_line.append(el[1:])
            s = int(list_line[0])-1
            d = int(list_line[1])-1
            # list line té 3 llocs: numspin, (1 o 2) i el valor de J
            j[s, d] = list_line[-1]
    return j

In [15]:
# get neighbours

def get_spin_neighbours(i, l):
    column = int(i % l)
    row = int(i / l)
    right = int(row * l + (column + 1)%l)
    up = int(((row + 1)%l) * l + column)
    left = int(row * l + (column - 1)%l)
    down = int(((row - 1)%l) * l + column)
    return np.array([right, up, left, down])


# Compute array index, spins position
def get_coupled_spins_from_index(i, l):
    """
    Given an index of j and the length of the square, computes
    the position of the paired spins.
    :param i: Int: Index of the j array (0,...,2N-1).
    :param l: Int: Length of the square (sqrt(N)).
    :return: Int tuple: Spin number of coupled spins    
    """
    e1 = int(i/2)
    e2 = int(i%2)
    sp1 = e1
    f1 = int(sp1 / l)
    c1 = int(sp1 % l)
    if e2 == 0:
        f2 = f1
        c2 = int((c1 + 1) % l)
    else:
        f2 = int((f1 - 1) % l)
        c2 = c1
    sp2 = int(f2 * l + c2)
    return sp1, sp2

def get_array_position_from_spin_direction(s, c, l):
    """
    Given a spin and a direction (up, down, right, left), computes the
    index of the j array of the paired spins.
    :param s: Int: Spin number.
    :param c: Char: Direction of the coupling: 'u', 'd', 'r', 'l'.
    :param l: Int: Length of the square (sqrt(N)).
    :return: Int: Index of the j array.
    """
    column = int(s % l)
    row = int(s / l)
    if c == 'r':
        return 2*s
    elif c == 'u':
        return 2*s + 1
    elif c == 'l':
        column = int((column - 1) % l)
        return 2 * int(row * l + column)
    else:
        row = int((row + 1)%l)
        return 2 * int(row * l + column) + 1

def get_spin_index_from_coupled(s, c, l):
    """
    Given a spin and a direction (up, down, right, left), computes the
    index of the neighbour spin.
    :param s: Int: Spin number.
    :param c: Char: Direction of the coupling: 'u', 'd', 'r', 'l'.
    :param l: Int: Length of the square (sqrt(N)).
    :return: Int: Index of the neightbour spin.
    """
    column = int(s % l)
    row = int(s / l)
    if c == 'r':
        column = int((column + 1) % l)
    elif c == 'u':
        row = int((row - 1)%l)
    elif c == 'l':
        column = int((column - 1) % l)
    else:
        row = int((row + 1)%l)
    return int(row * l + column)

In [17]:
for i in range(16):
    print('Node {}: {}'.format(i, get_spin_neighbours(i, 4)))

Node 0: [ 1  4  3 12]
Node 1: [ 2  5  0 13]
Node 2: [ 3  6  1 14]
Node 3: [ 0  7  2 15]
Node 4: [5 8 7 0]
Node 5: [6 9 4 1]
Node 6: [ 7 10  5  2]
Node 7: [ 4 11  6  3]
Node 8: [ 9 12 11  4]
Node 9: [10 13  8  5]
Node 10: [11 14  9  6]
Node 11: [ 8 15 10  7]
Node 12: [13  0 15  8]
Node 13: [14  1 12  9]
Node 14: [15  2 13 10]
Node 15: [12  3 14 11]


In [44]:
# Pseudolikelihood calculation
def log_pseudolikelihood(data, h, j, recalculate = True, previous = 0, h_prev=None, j_prev=None, new_row = -1):
    """Sums the rows of the pseudolikelihood"""
    m, n = data.shape
    if recalculate:
        output = 0
        for i in range(n):
            output += log_pseudolikelihood_row(data, h, j, i)
        return -float(output)/m
    else:
        return previous + float(log_pseudolikelihood_row(data, h_prev, j_prev, new_row) - log_pseudolikelihood_row(data, h, j, new_row))/m
        


def log_pseudolikelihood_row(data, h, j, i):
    """
    Computes the pseudolikelihood for each row
    Assumes j[i,i] = 0 for all i
    """
    m, n = data.shape
    output = 0
    for mu in range(m):
        output += math.log(1 + data[mu, i] * math.tanh(h[i] + multiply_j_spin_neighbours(j, data[mu, :], i)))
    return output

def multiply_j_spin_neighbours(j, spins, i):
    l = int(math.sqrt(len(j)/2))
    out = 0
    out += j[get_array_position_from_spin_direction(i, 'u', l)] * spins[get_spin_index_from_coupled(i, 'u', l)]
    out += j[get_array_position_from_spin_direction(i, 'd', l)] * spins[get_spin_index_from_coupled(i, 'd', l)]
    out += j[get_array_position_from_spin_direction(i, 'r', l)] * spins[get_spin_index_from_coupled(i, 'r', l)]
    out += j[get_array_position_from_spin_direction(i, 'l', l)] * spins[get_spin_index_from_coupled(i, 'l', l)]
    return out

In [47]:
def hamiltonian(spin_configuration, h, j):
    hamiltonian = 0
    n, d = j.shape
    l = int(np.sqrt(n))
    for i in range(n):
        subsum = 0
        for k in range(d):
            print('({},{}):{} * {} * {}'.format(i, get_spin_neighbours(i, l)[k], j[i,k], spin_configuration[i], spin_configuration[get_spin_neighbours(i, l)[k]]))
            subsum += spin_configuration[get_spin_neighbours(i, l)[k]] * j[i, k]
        hamiltonian += spin_configuration[i] * subsum
    hamiltonian += np.dot(h, spin_configuration)
    return -hamiltonian

In [48]:
j = read_j_coef('bonds.dat', 16)
h = np.zeros(16)
spins = read_configurations('test.txt', 16)
#print(j)
print(spins[0])

print(hamiltonian(spins[0], h, j))


[ 1 -1  1  1  1 -1 -1  1  1 -1  1  1 -1  1  1  1]
(0,1):0.0466667 * 1 * -1
(0,4):-0.0866667 * 1 * 1
(1,2):-0.3733333 * -1 * 1
(1,5):-0.1533333 * -1 * -1
(2,3):0.0366667 * 1 * 1
(2,6):0.0066667 * 1 * -1
(3,0):0.7633334 * 1 * 1
(3,7):0.61 * 1 * 1
(4,5):0.3833334 * 1 * -1
(4,8):-0.28 * 1 * 1
(5,6):0.56 * -1 * -1
(5,9):0.1633333 * -1 * -1
(6,7):0.1266667 * -1 * 1
(6,10):-0.71 * -1 * 1
(7,4):0.9666667 * 1 * 1
(7,11):-0.86 * 1 * 1
(8,9):-0.59 * 1 * -1
(8,12):-0.2366667 * 1 * -1
(9,10):-0.4033333 * -1 * 1
(9,13):-0.7566667 * -1 * 1
(10,11):0.7133334 * 1 * 1
(10,14):0.4233333 * 1 * 1
(11,8):-0.2933334 * 1 * 1
(11,15):0.72 * 1 * 1
(12,13):-0.9666667 * -1 * 1
(12,0):-0.9966667 * -1 * 1
(13,14):0.9833333 * 1 * 1
(13,1):-0.6666667 * 1 * -1
(14,15):-0.17 * 1 * 1
(14,2):0.9 * 1 * 1
(15,12):0.03 * 1 * -1
(15,3):0.27 * 1 * 1
-10.3733333
